Set-Up

In [1]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join('..')))

from scripts import request, download, stats
from scripts.constants import Constants
from scripts.table import extraction

project_path = os.path.abspath(os.path.expandvars(Constants.PROJECT_PATH))
experiments_path = os.path.join(project_path, Constants.Directories.EXPERIMENTS)
extracted_tables_path = os.path.join(experiments_path, Constants.Directories.EXTRACTED_TABLE)

connection_info = request.extract_infos('../private.json')

Download Articles

In [ ]:
article_dir = "CS_Test"
articles_path = os.path.join(experiments_path, Constants.Directories.ARTICLES, article_dir)

json_file_name = f"{article_dir}.json"

In [ ]:
# Download articles as HTML files
search_query = '"Entity Matching"'
download.get_articles(search_query, articles_path)

In [ ]:
extraction.extract_and_save_tables(articles_path, extracted_tables_path, json_file_name)

Claim Extraction

In [16]:
output_dir = "label_output/caption_citation"
tables_file_name = 'CS_GroundTruth_Labeling_All_Values.json'
msgs_dir = "CS_Label/Caption_Citation"

output_path = os.path.join(experiments_path, Constants.Directories.OUTPUT, output_dir)
tables_file_path = os.path.join(extracted_tables_path, tables_file_name)
msgs_path = os.path.join(project_path, Constants.Directories.MESSAGES, msgs_dir)

In [17]:
extraction.check_extracted_data(tables_file_path)

{'size': 102,
 'no_missing_attr': 102,
 'missing_caption': 0,
 'missing_citations': 0,
 'missing_both': 0}

In [18]:
request.set_up_test_dir(output_path, tables_file_path, msgs_path)
test_info = request.get_test_info(output_path)

In [19]:
request.run_test(connection_info, test_info, num_thread=50, max_cycles=3)

	 Saved prompt at: /home/matteo/workspace/menicacci/claim-extraction/experiments/output/label_output/html/1/output/1308.3357_0.txt
Sending request for: [1308.3357_0]
	 Saved prompt at: /home/matteo/workspace/menicacci/claim-extraction/experiments/output/label_output/html/1/output/1808.04511_0.txt
Sending request for: [1808.04511_0]
	 Saved prompt at: /home/matteo/workspace/menicacci/claim-extraction/experiments/output/label_output/html/1/output/2309.06171_1.txt
Sending request for: [2309.06171_1]
	 Saved prompt at: /home/matteo/workspace/menicacci/claim-extraction/experiments/output/label_output/html/1/output/1908.09059_1.txt
Sending request for: [1908.09059_1]
	 Saved prompt at: /home/matteo/workspace/menicacci/claim-extraction/experiments/output/label_output/html/1/output/2309.06171_0.txt
Sending request for: [2309.06171_0]
	 Saved prompt at: /home/matteo/workspace/menicacci/claim-extraction/experiments/output/label_output/html/1/output/2104.09677_0.txt
Sending request for: [2104.096

Ground Truth

In [ ]:
gt_path = os.path.join(experiments_path, Constants.Directories.GROUND_TRUTH)
gt_file = os.path.join(gt_path, "table_labeled.ods")
gt_answer_path = os.path.join(gt_path, "labeling")

stats.write_ground_truth(gt_file, gt_answer_path)

In [ ]:
output_dir = os.path.join(experiments_path, Constants.Directories.OUTPUT, "label_output")
caption_dir = os.path.join(output_dir, "caption")
html_dir = os.path.join(output_dir, "html")
citation_dir = os.path.join(output_dir, "citation")
caption_citation_dir = os.path.join(output_dir, "caption_citation")

output_dirs = [
    (caption_dir, 1),
    (html_dir, 1),
    (citation_dir, 1),
    (caption_citation_dir, 1)
]

save_path = os.path.join(experiments_path, "comparison")
stats.compare_results(gt_answer_path, output_dirs, save_path)